In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers -Uqq

     |████████████████████████████████| 5.8 MB 27.6 MB/s 
     |████████████████████████████████| 182 kB 62.2 MB/s 
     |████████████████████████████████| 7.6 MB 73.6 MB/s 


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [5]:
import sys
print(sys.version)

3.8.15 (default, Oct 12 2022, 19:14:39) 
[GCC 7.5.0]


In [6]:
# Importing basic libtest.jsonles
import numpy as np
import pandas as pd
import json
import argparse


In [7]:

# Importing some tools for flattening 2d arrays to 1d
from functools import reduce
from operator import add


In [8]:

# Importing hugging face library for getting the transformers and tokenizers
from transformers import AutoTokenizer, BartForConditionalGeneration,PegasusForConditionalGeneration,AdamW,get_linear_schedule_with_warmup 


In [9]:

# Importing some pytorch classes and functions
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
from torch import cuda 

In [10]:
%%capture
!pip install pytorch-lightning

In [11]:

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint,EarlyStopping
from pytorch_lightning.plugins import DeepSpeedPlugin


In [12]:
%%capture
!pip install deepspeed

In [13]:
from deepspeed.ops.adam import FusedAdam

In [14]:
# !pip install wandb

In [15]:
# import wandb

In [16]:
!pip install rouge
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 7.8 MB/s 


In [17]:

# Importing libraries for evaluation
from rouge import Rouge
from bert_score import score

In [18]:


# Importing library for progress bar GUI
#from fastai.text.core import progress_bar
from tqdm import tqdm

In [19]:

# Importing library for parallization
from joblib import Parallel,delayed,Memory


In [20]:
#Importing library for warnings
#import warnings

#warnings.filterwarnings('ignore') # Ignore warnings in the notebook

device = 'cuda' if cuda.is_available() else 'cpu' # Enable gpu if exsited else use cpu


In [21]:
# wandb.login() 

In [25]:
!nvidia-smi

Fri Dec  2 15:20:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:

PATH_TO_TRAIN_DATA = "/content/drive/MyDrive/NLP Project/Data/Extractive/Train_With_EXTSumm"
train_data = pd.read_csv(PATH_TO_TRAIN_DATA, index_col=0)


In [27]:
PATH_TO_TEST_DATA = "/content/drive/MyDrive/NLP Project/Data/Extractive/Test_With_EXTSumm"
test_data = pd.read_csv(PATH_TO_TEST_DATA, index_col=0)

In [28]:
train_data.columns

Index(['Heading', 'Summary', 'Article', 'id', 'cleaned_summary',
       'Ext_Summary'],
      dtype='object')

In [29]:
test_data.columns

Index(['Heading', 'Article', 'id', 'cleaned_article', 'Ext_Summary'], dtype='object')

Reading Target values

In [30]:
train_data.applymap(str)

,Heading,Summary,Article,id,cleaned_summary,Ext_Summary
0,India opposes China's Belt and Road Initiative...,The name of all member countries except India ...,"At SCO, India refuses to back China's Belt and...",0,the name of all member countries except india ...,"modi, in presence of xi and russian president ..."
1,"UN urges for maximum restraint, invokes Simla ...","Pakistan termed the Indian action as ""unilater...","UN chief invokes Shimla Agreement, calls for '...",1,"pakistan termed the indian action as ""unilater...","un chief invokes shimla agreement, calls for m..."
2,"China, Pak to finalise deal to develop SEZ und...","""The agreement will be finalised between Khybe...","China, Pak to finalise deal to develop SEZ und...",2,"""the agreement will be finalised between khybe...","china, pak to finalise deal to develop sez und..."
3,"Covaxin effectively neutralises both Alpha, De...",The top health research institute said that an...,"Covaxin effectively neutralises both Alpha, De...",3,the top health research institute said that an...,the nih said results of two studies of blood s...
4,"Top White House officials buried CDC report, r...",The decision to shelve detailed advice from th...,"In this April 22, 2020, file photo President D...",4,the decision to shelve detailed advice from th...,)redfield’s emailed comments contradict the wh...
...,...,...,...,...,...,...
10047,Covid-19 vaccine not likely to be available by...,It was not likely for a coronavirus vaccine to...,Covid-19 vaccine not likely to be available by...,10047,it was not likely for coronavirus vaccine to b...,covid-19 vaccine not likely to be available by...
10048,"Jill Biden visits Europe, will meet with Ukrai...","After flying overnight from Washington, the fi...",US First lady Jill Biden meets U.S. troops dur...,10048,"after flying overnight from washington, the fi...",now biden is using her second solo overseas tr...
10049,Coronaviurus: 29 foreigners infected by in Chi...,Twenty-nine foreign nationals in China were in...,Coronaviurus: 29 foreigners infected by in Chi...,10049,twenty-nine foreign nationals in china were in...,"ding xiangyang, deputy secretary-general of t..."
10050,Pakistan Defence’s Twitter account suspended f...,"On Saturday, numerous Indian Twitter users com...",Pakistan was yet again embarrassed on Saturday...,10050,"on saturday, numerous indian twitter users com...","on saturday, numerous indian twitter users co..."


In [31]:
train_summ = list(train_data['Summary'])


In [32]:
test_data.columns

Index(['Heading', 'Article', 'id', 'cleaned_article', 'Ext_Summary'], dtype='object')

In [111]:
# Getting the train, and test data splitted using splitting_text function declared above
train_text,train_summ = list(train_data.Ext_Summary) , train_summ
test_text = list(test_data.Ext_Summary)
val_text,val_summ = train_text[:100], train_summ[:100]

In [112]:
len(train_text)

10052

In [113]:
len(train_summ)

10052

In [114]:
# len(val_text)

In [115]:
len(val_summ)

100

In [116]:
len(test_text)

2513

## Tokenization

### Tokenization Class

In [117]:
class TransformersBaseTokenizer:

    """Class for encoding and decoding given texts for transformers"""

    def __init__(
        self,
        pretrained_tokenizer,
        model_type='bart',
        **kwargs
        ):
        self._pretrained_tokenizer = pretrained_tokenizer
        self.max_seq_len = pretrained_tokenizer.model_max_length
        self.model_type = model_type
        self.pad_token_id = pretrained_tokenizer.pad_token_id

    def __call__(self, *args, **kwargs):
        return self

    def tokenizer(self, t):
        """Limits the maximum sequence length and add the special tokens"""

        if self.model_type == 'bart':
            
            CLS = self._pretrained_tokenizer.cls_token
            SEP = self._pretrained_tokenizer.sep_token
            
            tokens = \
                self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len
                - 2]
            tokens = [CLS] + tokens + [SEP]

        elif self.model_type == 'pegasus':
            eos = self._pretrained_tokenizer.eos_token
            tokens = \
                self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len
                - 2]
            tokens = tokens + [eos]
                    

        return tokens

    def _numercalise(self, t):
        """Convert text to there coressponding ids"""
        
        tokenized_text = self._pretrained_tokenizer(
                t,
                max_length=self.max_seq_len,
                return_tensors='pt',
                padding='max_length',
                truncation=True,
                add_special_tokens=True,
                is_split_into_words=False,
                )
        return tokenized_text

    def _textify(self, input_ids):
        """Convert ids to thier coressponding text"""

        text = self._pretrained_tokenizer.batch_decode(input_ids,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=False)
        return text

    def _chunks(self, lst, n):
        """splitting the text into batches"""

        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    def numercalise(self, t, batch_size=4):
        """Convert text to thier coressponding ids and get the attention mask to differentiate between pad and input texts"""

        if isinstance(t, str):
            t = [t]  # convert str to list of str

        results = Parallel(n_jobs=-1)(delayed(self._numercalise)(batch)
                for batch in tqdm(list(self._chunks(t,
                batch_size))))
        input_ids = []
        attention_masks = []
        for i in results:
            input_ids.append(i['input_ids'])
            attention_masks.append(i['attention_mask'])

        return {'input_ids': torch.cat(input_ids),
                'attention_mask': torch.cat(attention_masks)}

    def textify(self, tensors, batch_size):
        """Convert ids to thier coressponding text"""

        if len(tensors.shape) == 1:
            tensors = [tensors]  # convert 1d tensor to 2d

        results = Parallel(n_jobs=-1, backend='threading'
                           )(delayed(self._textify)(summary_ids)
                             for summary_ids in
                             tqdm(list(self._chunks(tensors,
                             batch_size))))

        return reduce(add, results)

### Tokenizing data for BART Model

In [118]:
# Download the bart tokenizer from hugging face api
bart_tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-xsum')

In [119]:
# Passing the bart tokenizer to our TransformersBaseTokenizer wrapper
tokenizer = TransformersBaseTokenizer(bart_tokenizer)

In [120]:
len(test_text)

2513

In [121]:
# Converting the text into there coressponding input_ids and attention_mask to be interperted by the model

train_inputs = tokenizer.numercalise(train_text,16)

val_inputs = tokenizer.numercalise(val_text,16)

test_inputs = tokenizer.numercalise(test_text,16)

#train_outputs = tokenizer.numercalise(reduce(add,train_summ),16)

#val_outputs = tokenizer.numercalise(reduce(add,val_summ),16)

val_outputs = tokenizer.numercalise(val_summ,16)
train_outputs = tokenizer.numercalise(train_summ,16)


100%|██████████| 629/629 [00:29<00:00, 21.10it/s]


In [122]:
train_inputs['input_ids'].shape

torch.Size([10052, 1024])

In [123]:
train_inputs['attention_mask'].shape

torch.Size([10052, 1024])

In [124]:
# Getting the labels from train and val
labels = train_outputs['input_ids']
val_labels = val_outputs['input_ids']

In [125]:
train_outputs["input_ids"].shape

torch.Size([10052, 1024])

In [126]:
val_labels.shape

torch.Size([100, 1024])

In [127]:
train_inputs['labels'] = labels

In [128]:
train_inputs['labels'].shape

torch.Size([10052, 1024])

In [129]:
val_inputs['labels'] = val_labels

In [130]:
val_inputs['labels'].shape

torch.Size([100, 1024])

# Model Finetunning

In [131]:
hparams = argparse.Namespace()

hparams.freeze_encoder = True
hparams.freeze_embeds = True
hparams.eval_beams = 2

In [132]:
def shift_tokens_right(input_ids, pad_token_id):
    """ Shift input ids one token to the right, and wrap the last non pad token (usually <eos>).
      This is taken directly from modeling_bart.py
  """

    prev_output_tokens = input_ids.clone()
    index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1)
                    - 1).unsqueeze(-1)
    prev_output_tokens[:, 0] = input_ids.gather(1,
            index_of_eos).squeeze()
    prev_output_tokens[:, 1:] = input_ids[:, :-1]
    
    return prev_output_tokens

In [133]:
trial = [1,2,3,4,5,6]
trial[:-3]

[1, 2, 3]

In [134]:
def freeze_params(model):
    ''' Making the input part of the model as non trainable parameters'''    
    for layer in model.parameters():
      layer.requires_grad=False

In [135]:
class Model(pl.LightningModule):

    def __init__(
        self,
        lr,
        tokenizer,
        model,
        params,
        n_warmup_steps=None,
        n_training_steps =None
        ):

        super(Model, self).__init__()

        self.tokenizer = tokenizer
        self.model = model
        self.lr = lr
        self.params = params
        self.n_warmup_steps = n_warmup_steps
        self.n_training_steps = n_training_steps
        if self.params.freeze_encoder:
            freeze_params(self.model.get_encoder())

        if self.params.freeze_embeds:
            self.freeze_embeds()

    def freeze_embeds(self):
        ''' freeze the positional embedding parameters of the model '''

        freeze_params(self.model.model.shared)
        for d in [self.model.model.encoder, self.model.model.decoder]:
            freeze_params(d.embed_positions)
            freeze_params(d.embed_tokens)

    def forward(self, input_ids, **kwargs):
        return self.model(input_ids, **kwargs)

    def configure_optimizers(self):
        optimizer = FusedAdam(self.parameters(), lr=self.lr)
        scheduler = get_linear_schedule_with_warmup(
                                                      optimizer,
                                                      num_warmup_steps=self.n_warmup_steps,
                                                      num_training_steps=self.n_training_steps
                                                    )
        
        return dict(optimizer=optimizer,lr_scheduler=dict(scheduler=scheduler,interval='step'))

    def training_step(self, batch, batch_idx):

    # Load the data into variables

        src_ids, src_mask = batch[0], batch[1]
        tgt_ids = batch[2]

    # Shift the decoder tokens right (but NOT the tgt_ids)

        decoder_input_ids = shift_tokens_right(tgt_ids,
                self.tokenizer.pad_token_id)

    # Run the model and get the logits

        outputs = self(src_ids, attention_mask=src_mask,
                       decoder_input_ids=decoder_input_ids,
                       use_cache=False)
        labels_logits = outputs[0]

    # Create the loss function

        ce_loss_fct = \
            torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)

    # Calculate the loss on the un-shifted tokens

        loss = ce_loss_fct(labels_logits.view(-1,
                           labels_logits.shape[-1]), tgt_ids.view(-1))

        self.log(
            'train_loss/epoch',
            loss,
            prog_bar=True,
            logger=True,
            on_epoch=True,
            on_step = False
            )
        
        self.log(
            'train_loss/step',
            loss,
            prog_bar=True,
            logger=True,
            on_epoch=False,
            on_step = True
            )

        return {"loss": loss}

    def validation_step(self, batch, batch_idx):

    # Load the data into variables

        src_ids, src_mask = batch[0], batch[1]
        tgt_ids = batch[2]
        
        decoder_input_ids = shift_tokens_right(tgt_ids,
                self.tokenizer.pad_token_id)

    # Run the model and get the logits

        outputs = self(src_ids, attention_mask=src_mask,
                       decoder_input_ids=decoder_input_ids,
                       use_cache=False)
        labels_logits = outputs[0]

    # Create the loss function

        ce_loss_fct = \
            torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)

    # Calculate the loss on the un-shifted tokens

        loss = ce_loss_fct(labels_logits.view(-1,
                           labels_logits.shape[-1]), tgt_ids.view(-1))
        self.log(
            'val_loss/epoch',
            loss,
            prog_bar=True,
            logger=True,
            on_epoch=True,
            on_step = False,
            )
        
        self.log('val_loss/step',loss,prog_bar=True,logger=True,on_epoch=False,on_step = True)

        return {'loss': loss}
    
    def _chunks(self, lst, n):
        """splitting the text into batches"""

        for i in range(0, len(lst['input_ids']), n):
            yield lst['input_ids'][i:i + n],lst['attention_mask'][i:i + n]

    def _generate_text(
        self,
        text,
        mask,
        eval_beams,
        early_stopping=True,
        max_len=150,
        penalty_length = 0.2
    ):
        
        generated_ids = self.model.generate(
            text.to(device),
            attention_mask=mask.to(device),
            use_cache=True,
            decoder_start_token_id=self.tokenizer.pad_token_id,
            num_beams=eval_beams,
            max_length=max_len,
            early_stopping=early_stopping,
            length_penalty = penalty_length,
            no_repeat_ngram_size=3
            
            )
        
        return [self.tokenizer.decode(w, skip_special_tokens=True,
                clean_up_tokenization_spaces=True) for w in
                generated_ids]

    def generate_text(
        self,
        text,
        eval_beams,
        early_stopping=True,
        max_len=50,
        batch_size=2,
        length_penalty = 0.2
        ):
        ''' Function to generate text '''
        summaries = []
        
        for ids,mask in tqdm(list(self._chunks(text,batch_size))):
            txt = self._generate_text(ids,mask=mask,eval_beams = eval_beams,early_stopping = early_stopping,max_len = max_len,penalty_length = length_penalty)
            print("Summary gen\n")
            print(txt)
            print("\n")
            summaries.extend(txt)
    
        return summaries
            


    def save(self, model_name):
        model_extension =  model_name + '.h5'
        torch.save(self.model,model_extension)
        print ('Model is saved')
        return './'+  model_extension

In [136]:
# Create a dataloading module as per the PyTorch Lightning Docs

class SummaryDataModule(pl.LightningDataModule):

    def __init__(
        self,
        train,
        val=None,
        test=None,
        batch_size=2,
        ):
        super().__init__()
        self.train = train
        self.val = val
        self.test = test
        self.batch_size = batch_size

  # Load the training, validation and test sets in Pytorch Dataset objects

    def train_dataloader(self):
        dataset = TensorDataset(self.train['input_ids'],
                                self.train['attention_mask'],
                                self.train['labels'])
        train_data = DataLoader(dataset,
                                sampler=RandomSampler(dataset),
                                batch_size=self.batch_size)
        return train_data

    def val_dataloader(self):
        dataset = TensorDataset(self.val['input_ids'],
                                self.val['attention_mask'],
                                self.val['labels'])
        val_data = DataLoader(dataset, batch_size=self.batch_size)
        return val_data

    def test_dataloader(self):
        dataset = TensorDataset(self.test['input_ids'],
                                self.test['attention_mask'],
                                self.test['labels'])
        test_data = DataLoader(dataset, batch_size=self.batch_size)
        return test_data

### BART Finetunning

In [137]:
# Load the data into the model for training
summary_data = SummaryDataModule(train = train_inputs, val=val_inputs,
                                 batch_size=  1)

In [84]:
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-xsum")

In [85]:
steps_per_epoch=len(train_text) // 1
total_training_steps = steps_per_epoch * 5
warmup_steps = total_training_steps // 5

In [86]:
model = Model(lr = 3e-5, tokenizer = bart_tokenizer, model = bart_model, params = hparams,  n_warmup_steps=warmup_steps,
  n_training_steps=total_training_steps)

In [ ]:
# checkpoint = ModelCheckpoint(verbose=True,save_top_k = 1,monitor = 'val_loss/epoch',mode = 'min',filename='scitldr-{epoch:02d}-{val_loss:.2f}')
# wandb_logger = WandbLogger(project='Abstractive Text Summarization Filtered')

In [87]:
early_stopping_callback = EarlyStopping(monitor='val_loss/epoch', patience=3)

In [88]:
from pytorch_lightning.callbacks import TQDMProgressBar

In [89]:
from pytorch_lightning.loggers import CSVLogger

In [91]:
trainer = pl.Trainer(gpus=1,
                     precision=16,
                     max_epochs = 2,
                     auto_lr_find = False,
                     #callbacks=[checkpoint,early_stopping_callback],
                     #logger = wandb_logger
                    )

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [92]:
trainer.fit(model,summary_data)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Installed CUDA version 11.2 does not match the version torch was compiled with 11.3 but since the APIs are compatible, accepting this combination
Using /root/.cache/torch_extensions/py38_cu113 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py38_cu113/fused_adam...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py38_cu113/fused_adam/build.ninja...
Building extension module fused_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                         | Params
-------------------------------------------------------
0 | model | BartForConditionalGeneration | 406 M 
-------------------------------------------------------
201 M     Trainable params
204 M     Non-trainable params
406 M     Total params
812.581   Total estimated model params size (MB)


Loading extension module fused_adam...
Time to load fused_adam op: 43.51144289970398 seconds


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=2` reached.


#### path = model.save('bart_model')

In [93]:
path = model.save('/content/drive/MyDrive/NLP Project/Results/bart_model_Abseta_2_epoch')

Model is saved


In [ ]:
!tar -zcvf '/content/drive/MyDrive/NLP Project/Results/bart_model_1_epoch.tar.gz' '/content/drive/MyDrive/NLP Project/Results/bart_model_1_epoch.h5'

tar: Removing leading `/' from member names
/content/drive/MyDrive/NLP Project/Results/bart_model_1_epoch.h5


In [ ]:
from IPython.display import FileLink
FileLink(r'/content/drive/MyDrive/NLP Project/Results/bart_model_1_epoch.tar.gz')

/content/drive/MyDrive/NLP Project/Results/bart_model_1_epoch.tar.gz

In [138]:
def generate_summary(seed_line, model_,num_beam = 4,penalty_length =0.2):

  # Put the model on eval mode

    model_.to(torch.device('cuda'))
    model_.eval()

    line = model_.generate_text(seed_line, eval_beams=num_beam,length_penalty = penalty_length)

    return line

# Evaluation

In [ ]:
!gdown --id 19TvvqlPk7UxLGO4M1V5IEQWCef0_veP3
!tar -xvzf '/content/drive/MyDrive/NLP Project/Results/bart_model_1_epoch.tar.gz'

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=19TvvqlPk7UxLGO4M1V5IEQWCef0_veP3 

content/drive/MyDrive/NLP Project/Results/bart_model_1_epoch.h5


In [95]:
# model = torch.load('content/drive/MyDrive/NLP Project/Results/bart_model_1_epoch.h5')
model = torch.load('/content/drive/MyDrive/NLP Project/Results/bart_model_Abseta_2_epoch.h5')


In [103]:
b_model = Model(lr = 2e-5, tokenizer = bart_tokenizer, model = model, params = hparams)

In [97]:
class Scorer:

    def __init__(
        self,
        generate_text,
        gold_text,
        article,
        ):
        self.generated = generate_text
        self.gold_text = gold_text
        self.article = article

    def rouge_score(self):
        rouge_scores = []
        rouge = Rouge()
        for i,summ in enumerate(self.gold_text):
            maxx  = {'rouge-1':{'f': 0}}
            for summary in summ:
                try:
                    scores = rouge.get_scores(self.generated[i], summary)
                except:
                    print(summary,i)
                    return 'no'
                rouge_f = scores[0]['rouge-1']['f']
                if maxx['rouge-1']['f'] <= rouge_f:
                        maxx = scores[0]
            rouge_scores.append(maxx)
        rouge_1_f = []
        rouge_2_f = []
        rouge_l_f = []
        for rouge in rouge_scores:
            rouge_1_f.append(rouge['rouge-1']['f'])
            rouge_2_f.append(rouge['rouge-2']['f'])
            rouge_l_f.append(rouge['rouge-l']['f'])
            
        
        return [np.mean(rouge_1_f),np.mean(rouge_2_f),np.mean(rouge_l_f)]
    
    def bert_score(self):
        bert_scores = []
        for i,summ in enumerate(self.gold_text):
            maxx  = -1000
            for summary in summ:
                P,R,F1 = score([self.generated[i]], [summary])
                    
                if maxx <= F1:
                        maxx = F1
            bert_scores.append(maxx)
        return np.mean(bert_scores)
            

In [98]:
#num_beams = [2,6]
#p_l = [0.2,0.6]


num_beams = [2]
p_l = [0.2]


In [ ]:
report = {'rouge-1':[],'rouge-2':[],'rouge-l':[],'num_beam':[],'length_penalty':[],'bert_score':[]}

In [ ]:
val_inputs

{'input_ids': tensor([[    0,   415,  2850,  ...,     1,     1,     1],
         [    0,   879,   834,  ...,     1,     1,     1],
         [    0,   611,  1243,  ...,     1,     1,     1],
         ...,
         [    0,   438,  2838,  ...,     1,     1,     1],
         [    0, 33501,  7566,  ...,     1,     1,     1],
         [    0, 39409, 14932,  ...,     1,     1,     1]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[    0,   133,   766,  ...,     1,     1,     1],
         [    0, 20700, 20295,  ...,     1,     1,     1],
         [    0,   113,   133,  ...,     1,     1,     1],
         ...,
         [    0,  4154,  1863,  ...,     1,     1,     1],
         [    0,   133,  1752,  ...,     1,     1,     1],
         [    0,  3048,  2648,  ...,     1,     1,    

In [141]:
len(val_summ)

100

In [ ]:
test_inputs

{'input_ids': tensor([[    0,  9178, 12648,  ...,     1,     1,     1],
         [    0, 33501,  7566,  ...,     1,     1,     1],
         [    0, 10232,     4,  ...,     1,     1,     1],
         ...,
         [    0,   757,  3917,  ...,     1,     1,     1],
         [    0, 35213,  5033,  ...,     1,     1,     1],
         [    0, 28752,  1269,  ...,     1,     1,     1]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

#evaluate on val set

In [139]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [140]:
from rouge import Rouge

In [142]:
scorer = Rouge()

In [143]:
for beam in num_beams:
    for length in p_l:
        bart_summ = generate_summary(val_inputs,b_model,num_beam=beam,penalty_length=length)
        bart_scorer = scorer.get_scores(val_summ, bart_summ, avg=True)
        print(bart_scorer)

  2%|▏         | 1/50 [00:01<00:53,  1.09s/it]

Summary gen

["Modi, in presence of Xi Jinping and Russian President Vladimir Putin, said India's commitment to connectivity projects was reflected in its involvement in the international North South Corridor project, development of the Chabahar port and Ashgab", 'UN chief Antonio Guterres on Thursday urged India and Pakistan to exercise "maximum restraint" and refrain from taking steps that could affect the status of Jammu and Kashmir, as he highlighted the Shimla Agreement which rejects any third']




  4%|▍         | 2/50 [00:02<00:50,  1.06s/it]

Summary gen

['A team of China Road and Bridge Corporation (CRBC) is expected to visit Pakistan by the end of December to finalise an agreement for the development of the Rashakai Special Economic Zone (SEZ) under the China', "India's Covaxin, developed by Bharat Biotech in collaboration with the Indian Council of Medical Research, effectively neutralises both Alpha and Delta variants of coronavirus, the US National Institute of Health has said."]




  6%|▌         | 3/50 [00:03<00:50,  1.07s/it]

...

Summary gen

['A drug which is already approved to treat several blood cancers, is associated with reduced respiratory distress and a reduction in the overactive immune response in COVID-19 patients, an advance that may lead to a potential therapeutic for novel', "About 18 election rallies by President Donald Trump are estimated to have lead to more than 30,000 confirmed cases of coronavirus and likely led to'more than 700 deaths', a new study by Stanford University researchers said, stressing that the"]




 98%|█████████▊| 49/50 [00:58<00:01,  1.01s/it]

Summary gen

['President Donald Trump said Wednesday that he has ordered the Navy to “shoot down and destroy” any Iranian gunboats that harass US ships, a week after the Navy reported a group of Iranian boats made “dangerous and harassing approaches', 'On Saturday, Macron confirmed that discussions had been launched with the Taliban on the humanitarian operations and the evacuation of vulnerable Afghans.']




100%|██████████| 50/50 [00:59<00:00,  1.20s/it]

Summary gen

['Addressing a press conference in Islamabad, Aurangzeb chastised the Pakistan premier for "pushing the country into inflation and poverty", saying he should immediately resign "as his incompetence is costing Pakistan a lot".', "A defiant Prime Minister Boris Johnson said Saturday that he would resist attempts to delay Britain's departure from the European Union beyond the end of the month after Parliament postponed a decision on whether to back his Brexit deal and ordered the UK to ask"]


{'rouge-1': {'r': 0.5224733380235832, 'p': 0.4502607454399228, 'f': 0.4742957474076542}, 'rouge-2': {'r': 0.4077329619789426, 'p': 0.3424317750234633, 'f': 0.36427123995353083}, 'rouge-l': {'r': 0.488887019506377, 'p': 0.42047470624803623, 'f': 0.4435227590198014}}


# Run on test data

In [104]:
bart_summ_test = generate_summary(test_inputs,b_model,num_beam=2,penalty_length=0.2)

  0%|          | 1/1257 [00:01<22:56,  1.10s/it]

Summary gen

["For the first time in months, people across England are meeting indoors at pubs, restaurants, cinemas, cinem's, gyms and elsewhere as coronavirus rules were relaxed this week.", "The Pakistan Muslim League-Nawaz (PML-N) has decided to nominate Shahid Khaqan Abbasi as interim prime minister candidate and if elected, he will run the government as Interim Prime Minister until Sharif's"]




  0%|          | 2/1257 [00:01<20:16,  1.03it/s]

Summary gen

['An Indian-origin pathologist has been accused of botching up some post-mortems at a UK hospital and is under a police investigation.', 'China on Sunday started the world’s biggest exercise of population count, conducted once in a decade, to document demographic changes in the most populous country.']




  0%|          | 3/1257 [00:02<20:39,  1.01it/s]

...

Summary gen

['The Arab League (AL) called on Arab states on Monday to reactivate a boycott of Israel, describing it as a peaceful resistance to press the Israeli nation to abide by international resolutions.', 'The assessment, published in the journal Shock Waves, found that the size of the explosion was the equivalent of between 500-1100 tonnes of TNT - around 120th of the size the atomic bomb that was used on Hiroshima on August 6']




100%|█████████▉| 1256/1257 [20:14<00:00,  1.08it/s]

Summary gen

['Police suspect that someone had brought the ammunition to the plot during the last couple of days and tried to hide them in a garbage.', 'The US Department of Defence (DOD) released its annual public assessment of China’s military capabilities.']




100%|██████████| 1257/1257 [20:14<00:00,  1.03it/s]

Summary gen

['Prime Minister Narendra Modi and Chinese President Xi Jinping are expected to hold a bilateral meeting on Tuesday, nearly a week after India and China announced a resolution of the 73-day-long Doklam standoff']




In [ ]:
pd.DataFrame(report)

In [105]:
len(bart_summ_test)

2513

2

In [ ]:
bart_summ_test

## Saving the BART summ for test data

In [106]:
list_of_idx = []
list_of_summ = []

for idx,summ in enumerate(bart_summ_test):
  list_of_idx.append(idx)
  list_of_summ.append(summ)


In [107]:
len(list_of_idx)

2513

In [108]:
res = pd.DataFrame({'Summary':list_of_summ,'id':list_of_idx})

In [109]:
res

,Summary,id
0,"For the first time in months, people across En...",0
1,The Pakistan Muslim League-Nawaz (PML-N) has d...,1
2,An Indian-origin pathologist has been accused ...,2
3,China on Sunday started the world’s biggest ex...,3
4,A massive fire raged through an overcrowded pr...,4
...,...,...
2508,The Arab League (AL) called on Arab states on ...,2508
2509,"The assessment, published in the journal Shock...",2509
2510,Police suspect that someone had brought the am...,2510
2511,The US Department of Defence (DOD) released it...,2511


In [110]:
res.to_csv("/content/drive/MyDrive/NLP Project/Results/BART_ExtTEXT_2Epoch_test_results.csv")